In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.optim as optim

# Carregamento dos dados

In [44]:
data_set = pd.read_csv("./CompiledMeanDataRounded.csv", decimal=",", parse_dates=["Data"], infer_datetime_format=True).drop_duplicates()
data_set.head()

,Data,% Iron Feed,% Silica Feed,Starch Flow,Amina Flow,Ore Pulp Flow,Ore Pulp pH,Ore Pulp Density,Flotation Column 01 Air Flow,Flotation Column 02 Air Flow,...,Flotation Column 07 Air Flow,Flotation Column 01 Level,Flotation Column 02 Level,Flotation Column 03 Level,Flotation Column 04 Level,Flotation Column 05 Level,Flotation Column 06 Level,Flotation Column 07 Level,% Iron Concentrate,% Silica Concentrate
0,2017-03-29 12:00:00,60.18,9.35,1063.45,379.6,400.97,9.53,1.55,200.04,195.57,...,249.99,755.38,728.0,862.04,477.46,452.56,478.22,469.82,66.45,1.37
1,2017-03-29 13:00:00,60.18,9.35,2032.86,322.95,400.47,9.71,1.54,199.99,195.13,...,250.07,848.67,777.59,869.45,483.95,469.64,471.67,462.91,66.58,1.43
2,2017-03-29 14:00:00,60.18,9.35,1426.1,474.89,399.15,9.69,1.65,200.03,195.58,...,250.11,852.0,776.32,879.69,455.97,453.18,447.52,453.63,66.64,1.33
3,2017-03-29 15:00:00,60.18,9.35,616.73,395.13,398.94,9.86,1.56,199.94,195.61,...,250.04,855.89,780.4,882.1,449.4,448.52,450.87,448.41,66.4,1.3
4,2017-03-29 16:00:00,59.54,9.56,1369.26,318.08,400.82,9.94,1.54,199.89,196.25,...,250.12,851.58,784.47,884.86,450.22,451.9,451.58,449.39,63.65,5.48


In [45]:
data_set = data_set.to_numpy()
print(data_set.shape)

(3948, 24)


## Normalização dos dados

# Divisão do dados em treinamento e teste

In [42]:
# Separando em conjunto de treinamento e teste
training_data = data_set[:2775,:]
test_data = data_set[2775:,:]

# Implementação do Modelo

In [ ]:
class LSTMPredictor(nn.Module):
    def __init__(self, n_hidden, input_size):
        super(LSTMPredictor, self).__init__()
        self.input_size = input_size
        self.n_hidden = n_hidden

        self.lstm1 = nn.LSTMCell(self.input_size, self.n_hidden )
        self.lstm2 = nn.LSTMCell(self.n_hidden , self.n_hidden )
        self.lstm3 = nn.LSTMCell(self.n_hidden , self.n_hidden )
        self.linear = nn.Linear(self.n_hidden , 1)

    def forward(self, input):
        # N, 32
        outputs = []
        n_samples = input.size(0)

        h_t = torch.zeros(n_samples, self.n_hidden , dtype=torch.float64)
        c_t = torch.zeros(n_samples, self.n_hidden , dtype=torch.float64)
        h_t2 = torch.zeros(n_samples, self.n_hidden , dtype=torch.float64)
        c_t2 = torch.zeros(n_samples, self.n_hidden , dtype=torch.float64)

        for input_t in input.split(1, dim=1):
            h_t, c_t = self.lstm1(input_t, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs.append(output)
        
        outputs = torch.cat(outputs, dim=1)
        return outputs